In [2]:
import random
random.seed(1234)
import numpy as np
np.random.seed(1234)
import tensorflow as tf
tf.set_random_seed(1234)

In [115]:
### Update Input
tf.reset_default_graph() # This does not reset the random sequence!
with tf.device('/cpu:0'), tf.Session() as sess:
    x = tf.placeholder(tf.int32, [2, 6], name='x')
    # 20 words, each word has 2 definitions
    d = tf.placeholder(tf.int32, [48, 3], name='d')
    l = tf.placeholder(tf.int32, [48], name='sq_len')
    with tf.variable_scope('m'):
        emb = tf.get_variable('emb', [20, 6])
        # input sequences
        xemb = tf.nn.embedding_lookup(emb, x)
        xinputs = [tf.squeeze(_x, [1]) for _x in tf.split(xemb, 6, 1)]
        # feature sequences
        demb = tf.nn.embedding_lookup(emb, d)
        dinputs = [tf.squeeze(_x, [1]) for _x in tf.split(demb, 3, 1)]
        # feature LSTM
        dcell = tf.nn.rnn_cell.BasicLSTMCell(6)
        dcell_stack = tf.nn.rnn_cell.MultiRNNCell([dcell])
        dinit_state = dcell_stack.zero_state(48, tf.float32)
        doutputs, dstate = tf.nn.rnn(dcell_stack, dinputs, 
                                   initial_state=dinit_state, sequence_length=l)
        doutputs = tf.reshape(tf.concat(doutputs, 1), [-1, 6])
        idx = tf.range(0, 48) * 3 + (l - 1)
        dfinal_h = tf.gather(doutputs, idx)
        # input LSTM
        xcell = tf.nn.rnn_cell.BasicLSTMCell(6)
        xcell_stack = tf.nn.rnn_cell.MultiRNNCell([xcell] * 2)
        xinit_state = xcell_stack.zero_state(2, tf.float32)
        xstate = xinit_state
        xoutputs = []
        att_w = tf.get_variable('att_w', [6 + 6, 1], tf.float32)
        att_b = tf.get_variable('att_b', [1], tf.float32)
        atts = []
        uxdhs = []
        att_uxdhs = []
        prev_xhs = []
        for time_step in range(6):
            if time_step > 0: 
                tf.get_variable_scope().reuse_variables()
#                 previous_xh = xoutputs[time_step - 1]
            cur_x = tf.reshape(xinputs[time_step], [2, 1, -1])
            # need a flexible way to access this
            cur_dh = tf.reshape(
                tf.slice(dfinal_h, [4 * 2 * time_step, 0], [8, -1]),
                [2,4,-1])
            uxdh = tf.concat([cur_x, cur_dh], 1)
            
            prev_xh = tf.reshape(xstate[1][1], [2, 1, -1])
#             uxdh = tf.concat([uxdh, prev_xh], 2)
            uxdh_flat = tf.reshape(uxdh, [-1, 6])
            scores = tf.matmul(uxdh_flat, att_w) + att_b
            scores = tf.reshape(scores, [2, -1])
            att = tf.nn.softmax(scores)
            att_uxdh = tf.multiply(uxdh, tf.reshape(att, [2, -1, 1]))
            final_xinput = tf.reduce_sum(att_uxdh, 1)

            prev_xhs.append(prev_xh)
            uxdhs.append(uxdh)
            atts.append(att)
            att_uxdhs.append(att_uxdh)
            (cell_output, xstate) = xcell_stack(final_xinput, xstate)
            xoutputs.append(cell_output)
    sess.run(tf.global_variables_initializer())
    a = np.zeros((2,6), np.int32)
    b = np.zeros((48,3), np.int32)
    sl = np.random.randint(1, 4, (48))
    res = sess.run([dfinal_h, doutputs, uxdhs, atts, att_uxdhs, xoutputs, prev_xhs], 
                   feed_dict={x:a, d:b, l:sl})

ValueError: Dimensions must be equal, but are 6 and 12 for 'm/MatMul' (op: 'MatMul') with input shapes: [10,6], [12,1].

In [120]:
with tf.device('/cpu:0'), tf.Session() as sess:
    t1 = np.reshape(np.arange(3*4*5), [3,4,5])
    t2 = np.reshape(np.arange(3*1*2), [3,2])
    x = tf.Variable(t1, dtype=tf.float32)
    y = tf.Variable(t2, dtype=tf.float32)
    z = tf.Variable(np.ones([1, 4, 1]), dtype=tf.float32)
    y = tf.reshape(y, [3, 1, 2]) * z
    o = tf.concat([x,y], 2)
    _v = tf.Variable(1)
    tf.scalar_summary('tmp', _v)
    summary_op = tf.merge_all_summaries()
    sess.run(tf.global_variables_initializer())
    res = sess.run([x,y,z,o,None,summary_op])

TypeError: Fetch argument None has invalid type <type 'NoneType'>

In [118]:
res[4]

'\n\n\n\x03tmp\x15\x00\x00\x80?'

In [84]:
t3 = np.reshape(t2, [2, 1, 3]) * np.ones([1, 2, 1])

In [85]:
t3.shape

(2, 2, 3)

In [86]:
t3

array([[[ 5.,  5.,  5.],
        [ 5.,  5.,  5.]],

       [[ 6.,  6.,  6.],
        [ 6.,  6.,  6.]]])

In [107]:
t1 = np.reshape(np.arange(3*4*5), [3,4,5])
t2 = np.reshape(np.arange(3*1*2), [3,2])

In [109]:
t1

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])